In [1]:
import sys
import SimpleITK as sitk
import json
from glob import glob 
import os
from tqdm import tqdm
import numpy as np
import torch
sys.path.append("/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/src/utils")
from revert_normalisation import get_ct_normalisation_values, revert_normalisation
from image_metrics import ImageMetrics,ImageMetricsCompute
from test_analysis import result_analysis
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:


ct_plan_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset201_synthrad2025_task1_CT/nnUNetPlans.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
pred_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results/Dataset200_synthrad2025_task1_MR/nnUNetTrainerMRCT__nnUNetPlans__3d_fullres/fold_0/validation"
# revert_normalisation(pred_path, ct_mean, ct_std, save_path=pred_path + "_revert_norm")


CT mean: -726.3590698242188, CT std: 444.69049072265625


In [5]:
def result_analysis(input_path, raw_data_path):
    """
    Analyze the results of the predictions.
    """
    pred_paths = sorted(glob(os.path.join(input_path, '*.mha')))
    gt_path = os.path.join(raw_data_path, "gt_segmentations")
    mask_path = os.path.join(raw_data_path, "masks")
    testing_metrics = ImageMetricsCompute()
    testing_metrics.init_storage(["mae", "psnr", "ms_ssim"])

    for pred_path in tqdm(pred_paths):
        filename = os.path.basename(pred_path)
        gt_file = os.path.join(gt_path, filename)
        mask_file = os.path.join(mask_path, filename)

        img_pred = sitk.ReadImage(pred_path)
        img_gt = sitk.ReadImage(gt_file)
        img_mask = sitk.ReadImage(mask_file, sitk.sitkUInt8)

        array_pred = sitk.GetArrayFromImage(img_pred)
        array_gt = sitk.GetArrayFromImage(img_gt)
        array_mask = sitk.GetArrayFromImage(img_mask)

        res = testing_metrics.score_patient(array_gt, array_pred, array_mask)
        testing_metrics.add(res, filename)

    # aggregate results
    results = testing_metrics.aggregate()

    df = pd.DataFrame(
            {
                'patient_id': testing_metrics.storage_id,
                'mae': testing_metrics.storage['mae'],
                'ms_ssim': testing_metrics.storage['ms_ssim'],
                'psnr': testing_metrics.storage['psnr'],
            }
        )

    # save results, and df in the folder "results"
    results_path = os.path.join(input_path, "results.json")
    df_path = os.path.join(input_path, "results_individual.csv")
    if not os.path.exists(input_path):
        os.makedirs(input_path, exist_ok=True)
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=4)
    df.to_csv(df_path, index=False)
    return results, df

In [6]:
pred_path_revert_norm = pred_path + "_revert_norm"
raw_data_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset200_synthrad2025_task1_MR"
results, df = result_analysis(pred_path_revert_norm, raw_data_path)
print("mean mae:", results['mae']['mean'])
print("mean psnr:", results['psnr']['mean'])
print("mean ms_ssim:", results['ms_ssim']['mean'])


  0%|          | 0/116 [00:00<?, ?it/s]

100%|██████████| 116/116 [13:28<00:00,  6.97s/it]

mean mae: 109.02722691089096
mean psnr: 25.85192662986661
mean ms_ssim: 0.8442905581596798


In [ ]:
# save results, and df in the folder "results"
results_path = os.path.join(pred_path_revert_norm, "results.json")
df_path = os.path.join(pred_path_revert_norm, "results_df.csv")
os.makedirs(os.path.dirname(results_path), exist_ok=True)
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)
df.to_csv(df_path, index=False)